In [5]:
from pytket import Circuit
from tket.circuit import Tk2Circuit
from tket.protocol import CircuitReplacer,CircuitMatcher
from tket.matcher import CircuitUnit
from tket._tket.ops import TketOp
from tket.matcher import MatchOutcome, MatchContext
from tket.ops import TketOp as PyTketOp
import sys
sys.path.append("../../../tket2-rewriting/tket2_rewriting")
from label.label_map import init_label_map,update_label,get_label,get_map

In [6]:
orig_circuit = Circuit(2,2).CX(0,1).X(0).CX(0,1)
ops = list(Tk2Circuit(orig_circuit).to_tket1())
[(i, cmd.op.type.name, [q.index[0] for q in cmd.args]) for i, cmd in enumerate(ops)]


[(0, 'CX', [0, 1]), (1, 'X', [0]), (2, 'CX', [0, 1])]

In [7]:
from tket.ops import TketOp as PyTketOp

def is_type(op, name): return getattr(op, "type", None) and op.type.name == name
def is_type_or(op, name, py): 
    try: return is_type(op,name) or (op == py._to_rs())
    except: return is_type(op,name)

[(cmd.op.type.name, 
  is_type_or(cmd.op, "X", PyTketOp.X), 
  is_type_or(cmd.op, "CX", PyTketOp.CX))
 for cmd in ops]


[('CX', False, True), ('X', True, False), ('CX', False, True)]

In [8]:
labels = {0:"0", 1:"0"}
def flip(q): labels[q] = "1" if labels[q]=="0" else "0"

decisions = []
for i, cmd in enumerate(ops):
    name = cmd.op.type.name
    qs = [q.index[0] for q in cmd.args]
    if name == "X":
        flip(qs[0])
    elif name == "CX":
        c, t = qs
        decisions.append((i, "delete" if labels[c]=="0" else "keep", dict(labels)))
decisions


[(0, 'delete', {0: '0', 1: '0'}), (2, 'keep', {0: '1', 1: '0'})]